In [1]:
import urllib.request
import json
import pandas as pd
import numpy as np

https://opendata.cwb.gov.tw/dataset/observation/O-A0001-001

In [2]:
url = 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/{dataid}?Authorization={apikey}&format={data_type}'
dataid = 'O-A0001-001'
apikey = 'CWB-D90C66D1-BCC4-4839-ABAD-845A0DB5B140'
data_type = 'json'
link = url.format(dataid=dataid, apikey=apikey, data_type=data_type)
#link
link

'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/O-A0001-001?Authorization=CWB-D90C66D1-BCC4-4839-ABAD-845A0DB5B140&format=json'

In [3]:
#url path
urllib.request.urlretrieve(link,'{}.{}'.format(dataid, data_type))

('O-A0001-001.json', <http.client.HTTPMessage at 0x1fc70d69710>)

In [4]:
with open('{}.{}'.format(dataid, data_type), 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

In [5]:
weather_tmp = data['cwbopendata']['location']

In [9]:
# 把 value 就是值的 key 存起來 (除了 weatherElement 和 parameter )  
one_value_col = list(weather_tmp[0].keys())[:-2]
# 用動態變數的方式 建空 list 就可以不用打一堆字ㄌ 
for col in one_value_col :
    globals()[col] = []
    
# 存下 elementName 跟 parameterName 裡面指標的名字
weather_name = [i['elementName'] for i in weather_tmp[0]['weatherElement']]
parameter_name = [i['parameterName'] for i in weather_tmp[0]['parameter']]

weatherElement = []
parameter = []

for district in weather_tmp :
    # 也是用動態變數的方式新增觀測值
    for col in one_value_col :
        globals()[col].append(district[col])
    # 用 列表生成式 存裡面的值
    weatherElement.append([ i['elementValue']['value'] for i in district['weatherElement'] ])
    parameter.append([ i['parameterValue'] for i in district['parameter'] ])

In [10]:
#分 3 part 存dataframe 
# 1. 只有一個value的columns 2. weatherElement 裡面的指標跟觀測值 3. parameter 裡面的指標跟觀測值
a = pd.DataFrame({col : globals()[col] for col in one_value_col})
b = pd.DataFrame(weatherElement, columns=weather_name)
c = pd.DataFrame(parameter, columns=parameter_name)

In [11]:
#把 3個 dataframe 合併起來 
# ps 把 c 放前面只是為了讓前幾欄是地點的資訊
pd.concat([c, a, b], axis=1)

,CITY,CITY_SN,TOWN,TOWN_SN,lat,lon,lat_wgs84,lon_wgs84,locationName,stationId,...,HUMD,PRES,H_24R,H_FX,H_XD,H_FXT,D_TX,D_TXT,D_TN,D_TNT
0,臺北市,01,南港區,035,25.035950,121.611456,25.0341638888889,121.619680555556,國三南深路交流道,CM0010,...,0.81,1015.5,3.0,-99,-99,-99,26.50,2019-10-14T10:20:00+08:00,20.50,2019-10-14T21:20:00+08:00
1,新竹縣,10,竹北市,064,24.852500,120.998167,24.8507416666667,121.006377777778,水試所竹北,CM0100,...,0.64,1018.1,0.0,9.9,66,2019-10-14T23:00:00+08:00,27.90,2019-10-14T11:30:00+08:00,22.80,2019-10-14T22:10:00+08:00
2,彰化縣,14,鹿港鎮,140,24.091036,120.428836,24.0892916666667,120.436986111111,水試所鹿港,CM0110,...,0.57,1016.7,0.0,14.9,50,2019-10-14T22:08:00+08:00,28.80,2019-10-14T12:10:00+08:00,24.00,2019-10-14T06:50:00+08:00
3,雲林縣,16,臺西鄉,187,23.718408,120.183736,23.7166666666667,120.191686111111,水試所臺西,CM0120,...,0.58,1016.8,0.0,-99,-99,-99,27.70,2019-10-14T12:20:00+08:00,24.20,2019-10-14T23:00:00+08:00
4,臺南市,04,七股區,253,23.122550,120.071589,23.1208027777778,120.079675,水試所七股,CM0130,...,0.63,1016.6,0.0,-99,-99,-99,29.10,2019-10-14T15:40:00+08:00,24.70,2019-10-14T05:30:00+08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,嘉義縣,17,阿里山鄉,208,23.300847,120.656664,23.299075,120.664766666667,茶山,C0M800,...,0.95,968.2,0.0,-99,-99,-99,27.20,2019-10-14T15:20:00+08:00,22.30,2019-10-14T06:20:00+08:00
420,雲林縣,16,古坑鄉,196,23.688211,120.595289,23.68645,120.603416666667,棋山,C0K560,...,0.84,1004.6,0.0,-99,-99,-99,30.40,2019-10-14T14:20:00+08:00,21.80,2019-10-14T22:20:00+08:00
421,雲林縣,16,水林鄉,204,23.516064,120.221625,23.5143194444444,120.229736111111,蔦松,C0K550,...,0.68,1016.0,0.0,-99,-99,-99,30.50,2019-10-14T13:00:00+08:00,23.20,2019-10-14T23:00:00+08:00
422,臺南市,04,北門區,237,23.269517,120.117367,23.2677722222222,120.125480555556,北門,C0X290,...,0.66,1016.0,0.0,-99,-99,-99,29.10,2019-10-14T14:40:00+08:00,24.10,2019-10-14T04:00:00+08:00
